# PMU Disturbance Analysis - Predictive Modeling & Risk Scoring

Risk scores, survival analysis, and time series forecasting.

In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from src import predictive, temporal, visualizations as viz
import config

sns.set_style(config.PLOT_SETTINGS['style'])
print("Libraries loaded!")

## 1. Load Data

In [ ]:
from pathlib import Path
merged_df = pd.read_parquet(config.CLEANED_DATA)
pmu_df = pd.read_csv(Path(config.OUTPUT_DIR) / 'data' / 'pmu_data.csv')
disturbance_df = pd.read_csv(Path(config.OUTPUT_DIR) / 'data' / 'disturbance_data.csv')

# Parse dates
datetime_cols = merged_df.select_dtypes(include=['datetime64']).columns.tolist()
datetime_col = datetime_cols[0] if datetime_cols else 'DateTime'
print(f"Using datetime column: {datetime_col}")

## 2. Calculate Risk Scores for All 533 PMU Sections

In [ ]:
# Calculate composite risk scores
risk_scores = predictive.calculate_composite_risk_score(
    pmu_df,
    disturbance_df,
    section_col='SectionID',
    datetime_col=datetime_col
)

print(f"Risk scores calculated for {len(risk_scores)} sections")
print("\nRisk Category Distribution:")
print(risk_scores['Risk_Category'].value_counts())

print("\nTop 20 Highest Risk Sections:")
display(risk_scores.head(20)[['SectionID', 'Risk_Score_0_100', 'Risk_Category', 'Historical_Frequency']])

In [ ]:
# Plot risk score distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(risk_scores['Risk_Score_0_100'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Risk Score (0-100)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Risk Scores')
axes[0].grid(axis='y', alpha=0.3)

# Box plot by category
risk_scores.boxplot(column='Risk_Score_0_100', by='Risk_Category', ax=axes[1])
axes[1].set_xlabel('Risk Category')
axes[1].set_ylabel('Risk Score (0-100)')
axes[1].set_title('Risk Scores by Category')
plt.suptitle('')

plt.tight_layout()
viz.save_figure(fig, '05_01_risk_distribution')
plt.show()

## 3. Time Series Forecasting (ARIMA)

In [ ]:
# Aggregate daily counts
daily_counts = temporal.aggregate_disturbances_by_time(
    merged_df, datetime_col, freq='D'
)

# Forecast for 30, 60, 90 days
forecast_30 = predictive.forecast_arima(daily_counts, order=(1,1,1), forecast_periods=30)

print("30-Day Forecast:")
print(forecast_30['forecast'].head(10))
print(f"\nModel AIC: {forecast_30['aic']:.2f}")
print(f"Model BIC: {forecast_30['bic']:.2f}")

In [ ]:
# Plot forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Historical data
ax.plot(daily_counts.index[-90:], daily_counts.values[-90:], 
        label='Historical', color='steelblue', linewidth=2)

# Forecast
ax.plot(forecast_30['forecast'].index, forecast_30['forecast'].values,
        label='Forecast (30 days)', color='red', linewidth=2, linestyle='--')

# Confidence interval
ax.fill_between(
    forecast_30['confidence_interval'].index,
    forecast_30['confidence_interval'].iloc[:, 0],
    forecast_30['confidence_interval'].iloc[:, 1],
    alpha=0.3, color='red', label='95% CI'
)

ax.set_xlabel('Date')
ax.set_ylabel('Daily Disturbance Count')
ax.set_title('ARIMA Forecast: Next 30 Days', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
viz.save_figure(fig, '05_02_arima_forecast')
plt.show()

## 4. Section-Level Predictions

In [ ]:
# Predict disturbances for each section
predictions_30 = predictive.predict_section_disturbances(
    risk_scores, merged_df, forecast_days=30
)
predictions_60 = predictive.predict_section_disturbances(
    risk_scores, merged_df, forecast_days=60
)
predictions_90 = predictive.predict_section_disturbances(
    risk_scores, merged_df, forecast_days=90
)

# Combine predictions
all_predictions = predictions_30[['SectionID', 'Expected_Disturbances']].rename(
    columns={'Expected_Disturbances': 'Predicted_30d'}
)
all_predictions['Predicted_60d'] = predictions_60['Expected_Disturbances'].values
all_predictions['Predicted_90d'] = predictions_90['Expected_Disturbances'].values

print("Top 10 Sections by 30-Day Prediction:")
display(all_predictions.head(10))

## 5. Save Risk Scores and Predictions

In [ ]:
# Save risk scores
risk_scores.to_csv(config.RISK_SCORES, index=False)
print(f"Risk scores saved to: {config.RISK_SCORES}")

# Save predictions
all_predictions.to_csv(config.PREDICTIONS, index=False)
print(f"Predictions saved to: {config.PREDICTIONS}")

print("\nPredictive modeling complete!")

## Summary

- ✅ Calculated risk scores for all 533 PMU sections
- ✅ Generated 30/60/90-day forecasts
- ✅ Saved risk scores and predictions

**Next**: Notebook 06 (Operational Insights)